# Ćwiczenie: Fine-tuning prostego modelu odpowiedzi + rejestracja w AI Foundry

**Czas trwania:** 75 minut

**Cel ćwiczenia:** W tym ćwiczeniu przygotujemy zbiór danych do fine-tuningu, przeprowadzimy proces dostrajania modelu LLM, zarejestrujemy model w Azure AI Foundry oraz porównamy wyniki modelu podstawowego i dostrojonego.

## 1. Wprowadzenie do fine-tuningu modeli LLM

Fine-tuning (dostrajanie) modeli językowych to proces adaptacji wstępnie wytrenowanego modelu do specyficznego zadania lub domeny. Podczas gdy modele podstawowe, takie jak GPT-4 czy Llama 2, posiadają szeroką wiedzę ogólną, mogą nie być zoptymalizowane do konkretnych zastosowań w organizacji.

Korzyści z fine-tuningu:
- Dostosowanie modelu do specyficznego słownictwa i kontekstu organizacji
- Poprawa jakości odpowiedzi dla specjalistycznych zapytań
- Zmniejszenie "halucynacji" poprzez dostosowanie do faktów specyficznych dla organizacji
- Potencjalne zmniejszenie wielkości modelu przy zachowaniu jakości dla konkretnych zadań

W tym ćwiczeniu skupimy się na metodzie LoRA (Low-Rank Adaptation), która pozwala na efektywne dostrajanie dużych modeli bez konieczności modyfikowania wszystkich parametrów modelu.

##  Przygotowanie zbioru danych do fine-tuningu

Do fine-tuningu potrzebujemy zestawu par prompt-odpowiedź, które są specyficzne dla naszej domeny.

In [ ]:
# Przygotowanie przykładowego zbioru danych do fine-tuningu
# W rzeczywistym scenariuszu powinien to być specyficzny dla domeny zestaw pytań i odpowiedzi

# Przykładowy zestaw danych - pytania i odpowiedzi specyficzne dla organizacji
training_data = [
    {
        "prompt": "Co to jest Microsoft Fabric?",
        "completion": "Microsoft Fabric to zintegrowana platforma analityczna, która łączy różne usługi analityczne w jednym miejscu. Obejmuje Data Engineering, Data Factory, Synapse Data Science, Data Warehouse, Real-Time Analytics i Power BI. Fabric zapewnia zunifikowane środowisko do pracy z danymi, od inżynierii danych po raportowanie i analizę biznesową."
    },
    {
        "prompt": "Jak skonfigurować dostęp do Azure OpenAI w aplikacji DataBot?",
        "completion": "Aby skonfigurować dostęp do Azure OpenAI w aplikacji DataBot, musisz wykonać następujące kroki: 1) Utwórz zasób Azure OpenAI w portalu Azure, 2) Uzyskaj klucz API i endpoint, 3) Skonfiguruj zmienne środowiskowe OPENAI_API_KEY, OPENAI_API_BASE, OPENAI_API_TYPE i OPENAI_API_VERSION, 4) W kodzie aplikacji użyj tych zmiennych do inicjalizacji klienta OpenAI. Pamiętaj, żeby przechowywać klucze API w bezpiecznym miejscu, na przykład w Azure Key Vault lub Databricks Secrets."
    },
    {
        "prompt": "Jakie są zalety używania Databricks Lakehouse?",
        "completion": "Databricks Lakehouse oferuje wiele zalet: 1) Łączy możliwości hurtowni danych (strukturę, transakcje i jakość danych) z elastycznością jezior danych, 2) Zapewnia skalowalność i wydajność dzięki silnikowi Spark, 3) Umożliwia pracę z danymi strukturalnymi i niestrukturalnymi, 4) Oferuje zintegrowane narzędzia do uczenia maszynowego i analityki, 5) Zapewnia wsparcie dla różnych języków jak SQL, Python, R i Scala, 6) Umożliwia współpracę wielu użytkowników w czasie rzeczywistym, 7) Oferuje zaawansowane mechanizmy bezpieczeństwa i zarządzania dostępem do danych."
    },
    {
        "prompt": "Jak działa architektura RAG w kontekście naszej organizacji?",
        "completion": "W naszej organizacji, architektura RAG (Retrieval Augmented Generation) działa w następujący sposób: 1) Dane z różnych źródeł organizacyjnych (SharePoint, SQL, Data Lake) są indeksowane i przechowywane w bazie wektorowej, 2) Gdy użytkownik zadaje pytanie, system wyszukuje najbardziej odpowiednie fragmenty dokumentów, 3) Znalezione fragmenty są przekazywane do modelu LLM jako kontekst razem z pytaniem użytkownika, 4) Model generuje odpowiedź opartą na danych organizacyjnych, 5) System śledzi historię konwersacji, co pozwala na prowadzenie spójnych rozmów. Ta architektura zapewnia, że odpowiedzi modelu są oparte na aktualnych danych organizacyjnych, respektują polityki bezpieczeństwa i są zgodne z wewnętrznymi wytycznymi."
    },
    {
        "prompt": "Jak przygotować dane do fine-tuningu modelu LLM?",
        "completion": "Aby przygotować dane do fine-tuningu modelu LLM, należy: 1) Zebrać pary pytanie-odpowiedź reprezentatywne dla docelowego zastosowania, 2) Upewnić się, że dane są wysokiej jakości i poprawne merytorycznie, 3) Sformatować dane zgodnie z wymaganiami platformy do fine-tuningu (np. w formacie JSONL), 4) Podzielić dane na zbiór treningowy i walidacyjny, 5) Upewnić się, że rozkład tematów w danych jest zrównoważony, 6) Dla każdego promptu przygotować kompletną i dokładną odpowiedź, 7) Usunąć poufne lub wrażliwe informacje, które nie powinny być w modelu. W naszej organizacji używamy wewnętrznej dokumentacji, wiedzy eksperckiej i wcześniejszych interakcji z klientami jako źródeł danych treningowych."
    },
    {
        "prompt": "Jakie są podstawowe komendy Spark SQL używane w naszych projektach?",
        "completion": "W naszych projektach często używamy następujących komend Spark SQL: 1) CREATE TABLE - do tworzenia nowych tabel, 2) SELECT - do wybierania danych z tabel, 3) FROM - określanie źródła danych, 4) WHERE - filtrowanie wierszy, 5) GROUP BY - grupowanie danych, 6) JOIN - łączenie tabel, 7) ORDER BY - sortowanie wyników, 8) WITH - definiowanie tymczasowych podzapytań, 9) MERGE INTO - operacje upsert, 10) EXPLAIN - analiza planu wykonania zapytania. Pamiętaj, że w naszej organizacji preferujemy Delta Lake jako format do przechowywania danych, więc często używamy też specyficznych komend Delta jak OPTIMIZE, VACUUM czy DESCRIBE HISTORY."
    },
    {
        "prompt": "Jaka jest nasza procedura rejestrowania modeli ML w organizacji?",
        "completion": "Nasza procedura rejestrowania modeli ML obejmuje następujące kroki: 1) Udokumentowanie modelu (algorytm, hiperparametry, metryki, zbiór danych), 2) Przeprowadzenie testów modelu pod kątem dokładności i wydajności, 3) Rejestracja modelu w Azure ML Registry lub MLflow Registry z odpowiednimi tagami i metadanymi, 4) Przypisanie wersji modelu, 5) Dodanie opisu modelu w wewnętrznym katalogu modeli, 6) Przesłanie modelu do przeglądu przez zespół ds. zarządzania modelami, 7) Po zatwierdzeniu, oznaczenie modelu jako gotowego do produkcji, 8) Skonfigurowanie monitoringu modelu w środowisku produkcyjnym. Każdy model musi przejść ten proces przed wdrożeniem produkcyjnym."
    },
    {
        "prompt": "Jak skonfigurować połączenie do naszej bazy danych SQL z Databricks?",
        "completion": "Aby skonfigurować połączenie do naszej firmowej bazy danych SQL z Databricks, wykonaj następujące kroki: 1) Otwórz zakładkę 'Secrets' w Databricks, 2) Dodaj poświadczenia do zakresu 'company-db-credentials' (w tym 'jdbc-url', 'username' i 'password'), 3) W notebooku użyj następującego kodu: ```python\njdbc_url = dbutils.secrets.get('company-db-credentials', 'jdbc-url')\nusername = dbutils.secrets.get('company-db-credentials', 'username')\npassword = dbutils.secrets.get('company-db-credentials', 'password')\n\n# Połączenie z bazą danych\ndf = spark.read\n  .format('jdbc')\n  .option('url', jdbc_url)\n  .option('dbtable', 'schema.table_name')\n  .option('user', username)\n  .option('password', password)\n  .load()\n```\nPamiętaj, żeby nie przechowywać poświadczeń bezpośrednio w kodzie i zawsze używać mechanizmu Secrets. Jeśli masz problemy z dostępem, skontaktuj się z zespołem DBA."
    },
    {
        "prompt": "Jakie są nasze standardy dotyczące dokumentacji kodu?",
        "completion": "Nasze standardy dokumentacji kodu obejmują: 1) Każda funkcja i klasa musi mieć docstring w formacie NumPy/PyDoc, 2) Docstring powinien opisywać cel, parametry, zwracane wartości i wyjątki, 3) Skomplikowane fragmenty kodu powinny mieć komentarze wyjaśniające logikę, 4) Każdy moduł powinien zawierać opis swojego przeznaczenia na początku pliku, 5) Zmienne i funkcje powinny mieć opisowe nazwy, 6) Ważne algorytmy powinny mieć odniesienia do dokumentów projektowych lub literatury, 7) Zmiany w kodzie muszą być udokumentowane w pliku CHANGELOG.md, 8) Kod powinien zawierać przykłady użycia dla głównych funkcji, 9) Dokumentacja powinna być aktualizowana przy każdej istotnej zmianie kodu. Szczegółowe wytyczne znajdziesz w wewnętrznym repozytorium 'coding-standards'."
    },
    {
        "prompt": "Jak uzyskać dostęp do naszych wewnętrznych dashboardów analitycznych?",
        "completion": "Aby uzyskać dostęp do naszych wewnętrznych dashboardów analitycznych: 1) Otwórz przeglądarkę i przejdź do https://analytics.nasza-firma.com, 2) Zaloguj się używając swoich firmowych poświadczeń (SSO), 3) Po zalogowaniu będziesz miał dostęp do różnych folderów z raportami w zależności od swoich uprawnień, 4) Główne dashboardy znajdują się w folderze 'Corporate Dashboards', 5) Dashboardy departamentowe znajdują się w folderach z nazwami odpowiednich działów, 6) Jeśli potrzebujesz dostępu do dodatkowych raportów, wypełnij formularz 'Dashboard Access Request' dostępny w intranecie i uzyskaj zgodę właściciela danego dashboardu. W razie problemów technicznych, skontaktuj się z zespołem BI pod adresem bi-support@nasza-firma.com."
    }
]

# Konwersja na DataFrame
df_training = pd.DataFrame(training_data)
print(f"Przygotowano {len(df_training)} par pytanie-odpowiedź do fine-tuningu")

# Wyświetlenie kilku przykładów
print("\nPrzykłady:\n")
for i in range(min(3, len(df_training))):
    print(f"Prompt: {df_training.iloc[i]['prompt']}")
    print(f"Completion: {df_training.iloc[i]['completion'][:100]}...\n")

## 5. Podział danych na zbiory treningowy i walidacyjny

Aby kontrolować jakość fine-tuningu, podzielimy dane na zbiory treningowy i walidacyjny.

In [ ]:
# Podział danych na zbiory treningowy i walidacyjny
train_df, val_df = train_test_split(df_training, test_size=0.2, random_state=42)

print(f"Zbiór treningowy: {len(train_df)} przykładów")
print(f"Zbiór walidacyjny: {len(val_df)} przykładów")

# Zapisanie danych do plików JSONL (wymagany format dla fine-tuningu OpenAI)
train_jsonl_path = "train_data.jsonl"
val_jsonl_path = "val_data.jsonl"

# Funkcja pomocnicza do zapisywania w formacie JSONL
def df_to_jsonl(df, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            json_str = json.dumps({"prompt": row['prompt'], "completion": row['completion']})
            f.write(json_str + '\n')

# Zapisanie danych
df_to_jsonl(train_df, train_jsonl_path)
df_to_jsonl(val_df, val_jsonl_path)

print(f"Zapisano dane treningowe do {train_jsonl_path}")
print(f"Zapisano dane walidacyjne do {val_jsonl_path}")

## 6. Przygotowanie do fine-tuningu za pomocą MosaicML w Databricks

MosaicML to narzędzie w Databricks, które ułatwia fine-tuning modeli LLM.

In [ ]:
# Przygotowanie do fine-tuningu za pomocą MosaicML w Databricks
# Ta część kodu może wymagać dostosowania w zależności od dostępnej wersji MosaicML i Databricks

# Import wymaganych bibliotek
try:
    from transformers import TrainingArguments, Trainer, AutoModelForCausalLM, AutoTokenizer
    from datasets import load_dataset
    import torch
    print("Biblioteki do fine-tuningu zostały zaimportowane pomyślnie")
except ImportError as e:
    print(f"Błąd importu bibliotek: {str(e)}")
    print("Upewnij się, że zainstalowano wszystkie wymagane biblioteki")

# Sprawdzenie dostępności GPU
if torch.cuda.is_available():
    print(f"GPU jest dostępne: {torch.cuda.get_device_name(0)}")
    print(f"Liczba dostępnych GPU: {torch.cuda.device_count()}")
else:
    print("GPU nie jest dostępne. Fine-tuning może być bardzo powolny na CPU.")

## 7. Definicja parametrów fine-tuningu

Określimy teraz parametry procesu fine-tuningu, takie jak model bazowy i hiperparametry.

In [ ]:
# Definicja parametrów fine-tuningu

# Model bazowy - możesz użyć różnych modeli w zależności od dostępności
# Na potrzeby warsztatów używamy mniejszego modelu, który nie wymaga dużych zasobów
base_model_name = "databricks/dolly-v2-3b"  # Alternatywie: "EleutherAI/gpt-neo-1.3B" lub inny dostępny model

# Parametry treningu
training_args = TrainingArguments(
    output_dir="./fine-tuned-model",
    num_train_epochs=3,               # Liczba epok treningu
    per_device_train_batch_size=2,    # Rozmiar batcha (dostosuj do pamięci GPU)
    per_device_eval_batch_size=2,     # Rozmiar batcha dla walidacji
    warmup_steps=500,                 # Liczba kroków rozgrzewki
    weight_decay=0.01,                # Regularyzacja L2
    logging_dir="./logs",             # Katalog logów
    logging_steps=100,                # Co ile kroków logować
    evaluation_strategy="steps",      # Strategia ewaluacji
    eval_steps=500,                   # Co ile kroków ewaluować
    save_steps=1000,                  # Co ile kroków zapisywać model
    fp16=True,                        # Użycie precyzji połówkowej (FP16) dla przyspieszenia
    gradient_accumulation_steps=4,    # Akumulacja gradientów
    save_total_limit=2,               # Limit zapisanych modeli
    load_best_model_at_end=True,      # Załadowanie najlepszego modelu na końcu
    metric_for_best_model="eval_loss",# Metryka do wyboru najlepszego modelu
    greater_is_better=False           # Czy wyższa wartość metryki jest lepsza
)

print("Parametry fine-tuningu zostały zdefiniowane")

## 8. Przygotowanie modelu i tokenizera

Załadujemy model bazowy i tokenizer, które będą dostrajane.

In [ ]:
# Przygotowanie modelu i tokenizera
try:
    # Załadowanie modelu i tokenizera
    print(f"Ładowanie modelu {base_model_name}...")
    model = AutoModelForCausalLM.from_pretrained(base_model_name)
    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Upewnienie się, że tokenizer ma ustawiony token końca sekwencji
    if tokenizer.eos_token is None:
        tokenizer.eos_token = tokenizer.pad_token or "</s>"
    
    print("Model i tokenizer zostały załadowane pomyślnie")
    print(f"Liczba parametrów modelu: {model.num_parameters():,}")
except Exception as e:
    print(f"Błąd podczas ładowania modelu: {str(e)}")

## 9. Przygotowanie danych dla Transformers

Przygotujemy nasze dane w formacie odpowiednim dla biblioteki Transformers.

In [ ]:
# Przygotowanie danych dla Transformers
try:
    # Załadowanie danych z plików JSONL
    train_dataset = load_dataset('json', data_files=train_jsonl_path, split='train')
    val_dataset = load_dataset('json', data_files=val_jsonl_path, split='train')
    
    print(f"Załadowano {len(train_dataset)} przykładów treningowych i {len(val_dataset)} przykładów walidacyjnych")
    
    # Funkcja do tokenizacji danych
    def tokenize_function(examples):
        # Formatowanie promptów i odpowiedzi
        prompts = [f"Pytanie: {p}\nOdpowiedź:" for p in examples["prompt"]]
        completions = [f" {c}{tokenizer.eos_token}" for c in examples["completion"]]
        
        # Tokenizacja promptów
        tokenized_prompts = tokenizer(prompts, truncation=True, max_length=512)
        
        # Tokenizacja completions
        tokenized_completions = tokenizer(completions, truncation=True, max_length=512)
        
        # Przygotowanie pełnych tokenizowanych przykładów (prompt + completion)
        result = {
            "input_ids": [],
            "attention_mask": [],
            "labels": []
        }
        
        for prompt_ids, prompt_mask, compl_ids in zip(
            tokenized_prompts["input_ids"], 
            tokenized_prompts["attention_mask"],
            tokenized_completions["input_ids"]
        ):
            # Połączenie prompt i completion
            input_ids = prompt_ids + compl_ids[1:]  # Pomijamy pierwszy token z completion (zwykle jest to space)
            attention_mask = prompt_mask + [1] * (len(compl_ids) - 1)
            
            # Dla etykiet: -100 dla promptu (nie liczymy loss) i rzeczywiste ID tokenów dla completion
            labels = [-100] * len(prompt_ids) + compl_ids[1:]
            
            # Przycinanie do maksymalnej długości
            max_length = 1024
            if len(input_ids) > max_length:
                input_ids = input_ids[:max_length]
                attention_mask = attention_mask[:max_length]
                labels = labels[:max_length]
            
            result["input_ids"].append(input_ids)
            result["attention_mask"].append(attention_mask)
            result["labels"].append(labels)
        
        return result
    
    # Tokenizacja danych
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tokenized_val_dataset = val_dataset.map(tokenize_function, batched=True)
    
    print("Dane zostały przygotowane do fine-tuningu")
except Exception as e:
    print(f"Błąd podczas przygotowywania danych: {str(e)}")

## 10. Przeprowadzenie fine-tuningu

Teraz przeprowadzimy właściwy proces fine-tuningu modelu.

**Uwaga**: Ten proces może być czasochłonny i wymagać znacznych zasobów GPU.

In [ ]:
# Przeprowadzenie fine-tuningu
# UWAGA: Ten proces może być czasochłonny i wymagać znacznych zasobów GPU
# Na potrzeby warsztatów można ograniczyć liczbę epok lub użyć mniejszego modelu

try:
    # Inicjalizacja trenera
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train_dataset,
        eval_dataset=tokenized_val_dataset
    )
    
    print("Rozpoczynam fine-tuning...")
    # Uruchomienie treningu
    trainer.train()
    
    print("Fine-tuning został zakończony")
    
    # Zapisanie modelu
    fine_tuned_model_path = "./fine-tuned-model/final"
    trainer.save_model(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)
    
    print(f"Model został zapisany w {fine_tuned_model_path}")
except Exception as e:
    print(f"Błąd podczas fine-tuningu: {str(e)}")

## 11. Rejestracja modelu w Azure AI Foundry (Azure ML Model Registry)

Teraz zarejestrujemy dostrojony model w Azure ML Model Registry, aby można go było łatwo używać w różnych aplikacjach.

In [ ]:
# Rejestracja modelu w Azure ML Model Registry
try:
    # Ścieżka do zapisanego modelu
    model_path = fine_tuned_model_path
    
    # Ustawienie tagu dla modelu
    tags = {
        "base_model": base_model_name,
        "task": "question-answering",
        "domain": "enterprise-databot",
        "training_data_size": str(len(train_dataset)),
        "created_by": "workshop-participant"
    }
    
    # Rejestracja modelu
    model_name = "enterprise-databot-finetuned"
    model_description = "Fine-tuned model for enterprise DataBot based on " + base_model_name
    
    # Zarejestruj model w Azure ML
    registered_model = Model.register(
        workspace=ws,
        model_path=model_path,
        model_name=model_name,
        description=model_description,
        tags=tags
    )
    
    print(f"Model został zarejestrowany: {registered_model.name}, wersja: {registered_model.version}")
    print(f"ID modelu: {registered_model.id}")
except Exception as e:
    print(f"Błąd podczas rejestracji modelu: {str(e)}")

## 12. Testowanie dostrojonego modelu

Przetestujemy nasz dostrojony model i porównamy jego odpowiedzi z modelem bazowym.

In [ ]:
# Testowanie dostrojonego modelu
try:
    # Załadowanie dostrojonego modelu
    fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path)
    fine_tuned_tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)
    
    # Załadowanie modelu bazowego (dla porównania)
    base_model = AutoModelForCausalLM.from_pretrained(base_model_name)
    base_tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    
    # Przykładowe pytania do testowania
    test_questions = [
        "Co to jest Microsoft Fabric?",
        "Jak działa architektura RAG w kontekście naszej organizacji?",
        "Jakie są nasze standardy dokumentacji kodu?"
    ]
    
    # Funkcja do generowania odpowiedzi
    def generate_response(model, tokenizer, question):
        prompt = f"Pytanie: {question}\nOdpowiedź:"
        inputs = tokenizer(prompt, return_tensors="pt")
        
        # Przeniesienie na GPU, jeśli jest dostępne
        if torch.cuda.is_available():
            inputs = {k: v.to("cuda") for k, v in inputs.items()}
            model = model.to("cuda")
        
        # Generowanie odpowiedzi
        outputs = model.generate(
            inputs["input_ids"], 
            max_length=1024, 
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
        
        # Dekodowanie odpowiedzi
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Wyodrębnienie tylko części "Odpowiedź:"
        answer_start = response.find("Odpowiedź:")
        if answer_start != -1:
            response = response[answer_start + len("Odpowiedź:"):].strip()
        
        return response
    
    # Porównanie odpowiedzi
    print("\nPorównanie modelu bazowego i dostrojonego:\n")
    for question in test_questions:
        print(f"Pytanie: {question}")
        
        # Odpowiedź z modelu bazowego
        base_response = generate_response(base_model, base_tokenizer, question)
        print(f"\nOdpowiedź modelu bazowego:\n{base_response[:500]}..." if len(base_response) > 500 else f"\nOdpowiedź modelu bazowego:\n{base_response}")
        
        # Odpowiedź z dostrojonego modelu
        fine_tuned_response = generate_response(fine_tuned_model, fine_tuned_tokenizer, question)
        print(f"\nOdpowiedź dostrojonego modelu:\n{fine_tuned_response[:500]}..." if len(fine_tuned_response) > 500 else f"\nOdpowiedź dostrojonego modelu:\n{fine_tuned_response}")
        
        print("\n" + "-"*80 + "\n")
except Exception as e:
    print(f"Błąd podczas testowania modelu: {str(e)}")

## 13. Integracja dostrojonego modelu z DataBotem

Teraz zintegrujemy nasz dostrojony model z DataBotem stworzonym w poprzednim ćwiczeniu.

In [ ]:
# Integracja dostrojonego modelu z DataBotem
# UWAGA: Ten kod zakłada, że wcześniej zdefiniowano klasę DataBot z poprzedniego ćwiczenia
# Może wymagać dostosowania w zależności od implementacji DataBota

try:
    # Import potrzebnych klas z poprzedniego ćwiczenia
    # W rzeczywistym scenariuszu te klasy byłyby zaimportowane z odpowiednich modułów
    # Tutaj dla uproszczenia definiujemy podstawowy szkielet DataBota
    
    class SimpleDataBot:
        def __init__(self, model_path=None, use_fine_tuned=True):
            # Inicjalizacja tokenizera i modelu
            if use_fine_tuned and model_path:
                print(f"Ładowanie dostrojonego modelu z {model_path}...")
                self.model = AutoModelForCausalLM.from_pretrained(model_path)
                self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            else:
                print(f"Ładowanie modelu bazowego {base_model_name}...")
                self.model = AutoModelForCausalLM.from_pretrained(base_model_name)
                self.tokenizer = AutoTokenizer.from_pretrained(base_model_name)
            
            # Przeniesienie modelu na GPU, jeśli dostępne
            if torch.cuda.is_available():
                self.model = self.model.to("cuda")
                
            print("DataBot został zainicjalizowany")
        
        def process_query(self, query):
            # Przygotowanie promptu
            prompt = f"Pytanie: {query}\nOdpowiedź:"
            inputs = self.tokenizer(prompt, return_tensors="pt")
            
            # Przeniesienie na GPU, jeśli jest dostępne
            if torch.cuda.is_available():
                inputs = {k: v.to("cuda") for k, v in inputs.items()}
            
            # Generowanie odpowiedzi
            outputs = self.model.generate(
                inputs["input_ids"], 
                max_length=1024, 
                temperature=0.7,
                top_p=0.9,
                do_sample=True,
                pad_token_id=self.tokenizer.eos_token_id
            )
            
            # Dekodowanie odpowiedzi
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Wyodrębnienie tylko części "Odpowiedź:"
            answer_start = response.find("Odpowiedź:")
            if answer_start != -1:
                response = response[answer_start + len("Odpowiedź:"):].strip()
            
            return response
    
    # Inicjalizacja DataBota z dostrojonym modelem
    databot_fine_tuned = SimpleDataBot(model_path=fine_tuned_model_path, use_fine_tuned=True)
    
    # Testowanie DataBota
    test_question = "Jak skonfigurować połączenie do naszej bazy danych SQL z Databricks?"
    
    print(f"\nPytanie: {test_question}")
    response = databot_fine_tuned.process_query(test_question)
    print(f"\nOdpowiedź DataBota:\n{response}")
except Exception as e:
    print(f"Błąd podczas integracji z DataBotem: {str(e)}")

## 14. Monitorowanie i wersjonowanie modelu

Sprawdźmy informacje o zarejestrowanym modelu i zobaczmy, jak można zarządzać jego wersjami.

In [ ]:
# Monitorowanie i wersjonowanie modelu w Azure ML
try:
    # Pobranie zarejestrowanego modelu
    model_versions = Model.list(ws, name=model_name)
    
    print(f"\nZarejestrowane wersje modelu '{model_name}':")
    for model in model_versions:
        print(f"  - Wersja: {model.version}, Utworzono: {model.creation_time}, ID: {model.id}")
    
    # Dodanie nowej wersji modelu (symulacja)
    print("\nDodawanie nowych tagów do modelu...")
    latest_model = model_versions[0]  # Najnowsza wersja
    
    # Dodanie nowych tagów
    latest_model.add_tags({"evaluation_accuracy": "0.85", "status": "production-ready"})
    
    # Pobranie zaktualizowanych tagów
    updated_model = Model(ws, name=model_name, version=latest_model.version)
    
    print("\nZaktualizowane tagi modelu:")
    for tag_name, tag_value in updated_model.tags.items():
        print(f"  - {tag_name}: {tag_value}")
    
except Exception as e:
    print(f"Błąd podczas demonstracji wersjonowania modelu: {str(e)}")

## 15. Zadania do wykonania

1. Dodaj więcej przykładów do zbioru treningowego, aby poprawić jakość dostrojonego modelu
2. Zmodyfikuj parametry fine-tuningu (np. liczbę epok, rozmiar batcha) i zobacz, jak wpływa to na wyniki
3. Porównaj odpowiedzi modelu bazowego i dostrojonego na pytania specyficzne dla twojej organizacji
4. Zaimplementuj prostą metrykę oceny jakości odpowiedzi generowanych przez model

## Zadania dodatkowe

Jeśli masz więcej czasu, możesz spróbować:

1. Eksperymentowanie z różnymi parametrami fine-tuningu (liczba epok, rozmiar batcha, tempo uczenia)
2. Dodanie większej liczby przykładów do zbioru treningowego lub przygotowanie własnych przykładów
3. Implementacja mechanizmu ewaluacji jakości modelu z wykorzystaniem metryk takich jak BLEU, ROUGE itp.
4. Przygotowanie pipeline'u CI/CD do automatycznego fine-tuningu i wdrażania modelu